# Projet maison n° 3

In [1]:
# imports
import pandas as pd
import numpy as np
import os

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [44]:
# names us
# File documentation:
# Each file is sorted first on sex and then on number of occurrences in descending order
# When there is a tie on the number of occurrences, names are listed in alphabetical order
def df_names_us():
    
    # Our columns' names
    dfcolumns=['year', 'name', 'gender', 'births']
    dfcolumns_dtypes = [np.int32, str, str, np.int32]

    # We create a dict with empty lists to fill in later
    rawdata = {k:[] for k in dfcolumns}

    for filename in os.listdir('names'): # for each file in the dict
        if filename.endswith(".txt"):   
            file = pd.read_csv('names/'+filename, header=None, names=dfcolumns[1:]) # read the file
            file['year'] = int(filename[3:-4]) # we add a column with the year

            # we append the newly read data to the list
            # this method also ensures that we don't change the order in which the data is sorted
            for col in dfcolumns:
                rawdata[col] += list(file[col])
            
    # we create the data frame susing the list that we just generated
    df =  pd.DataFrame({k:rawdata[k] for k in dfcolumns})

    # we convert the columns in the correct types
    df = df.astype(dict(zip(dfcolumns, dfcolumns_dtypes)))

    # if we wanted to sort the values, but they are already sorted properly
    #df = df.sort_values(by=['year', 'gender', 'births', 'name'], ascending=[True, True, False, True])

    return df

In [45]:
names_us = df_names_us()
names_us

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
2020858,2020,Zykell,M,5
2020859,2020,Zylus,M,5
2020860,2020,Zymari,M,5
2020861,2020,Zyn,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [109]:
# names fr
def df_names_fr():
    # Our columns' names, the old and new ones
    df_new_cols=['year', 'name', 'gender', 'births']
    df_new_cols_dtypes = [np.int32, str, str, np.int32]
    df_old_cols=['annais', 'preusuel', 'sexe', 'nombre']
    
    # We read our data
    df = pd.read_csv('nat2020.csv', sep=';')

    # We rename the columns
    df = df.rename(columns=dict(zip(df_old_cols, df_new_cols)))

    # We change the columns' order
    df = df.reindex(columns=df_new_cols)

    # We convert the columns into the correct types
    df = df.astype(dict(zip(df_new_cols, df_new_cols_dtypes)), errors='ignore')

    # We replace the gender values
    df['gender'] = df['gender'].map({'2': 'F', '1': 'M'})

    # We remove names with fewer than 2 letters
    df = df[df['name'].str.len() >= 2]

    # We put the names into the same format as the names us data set
    df['name'] = df['name'].str.title()

    # We remove lines with unsable data
    df = df[df['year'] != 'XXXX']
    #  df[~pd.to_numeric(df['year'], errors='coerce').isna()]
    df = df[df['name'] != '_Prenoms_Rares']

    # We sort the lines and reset the index
    df = df.sort_values(by=['year', 'gender', 'births', 'name'], ascending=[True, True, False, True], ignore_index=True)
    
    return df

In [111]:
names_fr = df_names_fr()
names_fr

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696
...,...,...,...,...
630402,2020,Élouen,M,3
630403,2020,Énes,M,3
630404,2020,Éphraïm,M,3
630405,2020,Éros,M,3


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [ ]:
# taux de change
def df_taux_change(devises):
    df = pd.DataFrame([], columns=devises)
    return df

### Tests

In [100]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [112]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [113]:
# run tests
run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ERROR

ERROR: test_df_taux_change (__main__.Lesson4Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\Pierre\AppData\Local\Temp/ipykernel_9200/3530408247.py", line 31, in test_df_taux_change
    df = df_taux_change(['CHF', 'GBP', 'USD'])
NameError: name 'df_taux_change' is not defined

----------------------------------------------------------------------
Ran 3 tests in 4.927s

FAILED (errors=1)
